In [1]:
import tensorflow as tf
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)


2023-01-04 14:14:11.039562: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 14:14:11.039576: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow version: 2.9.1


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
        (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


2023-01-04 14:14:12.505231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 14:14:12.505489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 14:14:12.505522: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-04 14:14:12.505549: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-04 14:14:12.505576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
class BaseDNN1(tf.keras.Model):
    def __init__(self, node_counts, dropout_ratios=None, activation='relu'):
        super(BaseDNN1, self).__init__()
        self.node_counts = node_counts
        self.dropout_ratios = [0 for _ in range(len(self.node_counts))] if dropout_ratios is None else dropout_ratios
        
        self.activation = activation

        for i in range(len(self.node_counts)):
            n_nodes = self.node_counts[i]
            dropout_ratio = self.dropout_ratios[i]
            setattr(self, f'dense_{i}', layers.Dense(n_nodes, activation=self.activation))
            # setattr(self, f'dropout_{i}', layers.Dropout(dropout_ratio))

    def call(self, input_tensor):
        x = input_tensor
        for i in range(len(self.node_counts)):
            x = getattr(self, f'dense_{i}')(x)
            # x = getattr(self, f'dropout_{i}')(x)
        return x


class ConvolutionalLayers(tf.keras.Model):
    def __init__(self, filters, kernel_sizes, activation='relu', strides=None, paddings=None):
        super(ConvolutionalLayers, self).__init__()
        self.filters = filters
        self.kernels_sizes = kernel_sizes
        self.activation = activation

        if strides is None:
            self.strides = []
            for _ in range(len(filters)):
                self.strides.append((1, 1))
        else:
            self.strides = strides

        if paddings is None:
            self.paddings = []
            for _ in range(len(filters)):
                self.paddings.append("valid")
        else:
            self.paddings = paddings


        for i in range(len(self.filters)):
            n_filters = self.filters[i]
            kernel_size = self.kernels_sizes[i]
            stride = self.strides[i]
            padding = self.paddings[i]

            setattr(self, f'conv2d_{i}', layers.Conv2D(n_filters, kernel_size, strides=stride, padding=padding, activation=self.activation))

    def call(self, input_tensor):
        x = input_tensor
        for i in range(len(self.filters)):
            x = getattr(self, f'conv2d_{i}')(x)
        return x

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, loss_calculation, accuracy_calculation):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(model, images, labels, loss_object, optimizer, loss_calculation, accuracy_calculation):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

"""model_inputs = layers.Input(shape=(28, 28, 1), name="input_layer")
x = layers.Conv2D(32, 3, activation='relu')(model_inputs)
x = layers.Flatten()(x)
x = BaseDNN1([128, 10])(x)
model = tf.keras.Model(inputs=model_inputs, outputs=x)"""
model = tf.keras.Sequential([])

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, optimizer, test_loss, test_accuracy)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )


Epoch 1, Loss: 0.380532830953598, Accuracy: 86.0, Test Loss: 0.2966935336589813, Test Accuracy: 88.13999938964844
Epoch 2, Loss: 0.28414151072502136, Accuracy: 88.51333618164062, Test Loss: 0.28843846917152405, Test Accuracy: 88.45999908447266
Epoch 3, Loss: 0.2611176073551178, Accuracy: 89.12833404541016, Test Loss: 0.28167036175727844, Test Accuracy: 88.5999984741211
Epoch 4, Loss: 0.25282567739486694, Accuracy: 89.3183364868164, Test Loss: 0.2940133810043335, Test Accuracy: 88.36000061035156
Epoch 5, Loss: 0.2473232001066208, Accuracy: 89.47166442871094, Test Loss: 0.28504350781440735, Test Accuracy: 88.68000030517578


In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 flatten (Flatten)           (None, 21632)             0         
                                                                 
 base_dnn1 (BaseDNN1)        (None, 10)                2770314   
                                                                 
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.layers